In [1]:
import pandas as pd
import requests
import re

In [2]:
# STEP 1: Load Excel file with democracy-related questions
df = pd.read_excel('/content/drive/MyDrive/final_project/CHATBOT QNA.xlsx')
df.columns = df.columns.str.strip()

letters = ['A', 'B', 'C', 'D', 'E']
qa_pairs = []

print("🗳️ Welcome to the Democracy Personality Chatbot!\nPlease answer the following 10 questions:\n")


🗳️ Welcome to the Democracy Personality Chatbot!
Please answer the following 10 questions:



In [3]:
# STEP 2: Load Excel File
df = pd.read_excel('/content/drive/MyDrive/final_project/CHATBOT QNA.xlsx')
df.columns = df.columns.str.strip()



In [4]:


# STEP 2: Ask 10 questions and collect answers
for i in range(10):
    row = df.iloc[i]
    print(f"\nQ{i+1}: {row['Questions']}")
    for j in range(5):
        print(f"{letters[j]}) {row[f'Option {letters[j]}']}")

    while True:
        answer_letter = input("Your answer (A/B/C/D/E): ").strip().upper()
        if answer_letter in letters:
            break
        print("❗ Please enter A, B, C, D, or E.")

    answer_text = row[f'Option {answer_letter}']
    qa_pairs.append((row['Questions'], answer_text))


Q1: Do you feel it is your duty to participate in every election, even if you don't strongly support any candidate?"
A) Strongly Disagree
B) Disagree
C) Neutral
D) Agree
E) Strongly Agree  
Your answer (A/B/C/D/E): c
Your answer (A/B/C/D/E): 

Q2: Do you enjoy discussing political and social issues with others, even if they have different views?ocial issues with others, even if they have different views.
A) Strongly Disagree
B) Disagree
C) Neutral
D) Agree
E) Strongly Agree  
❗ Please enter A, B, C, D, or E.
Your answer (A/B/C/D/E): c

Q3: Do you often feel anxious or frustrated when thinking about the current political situation?
A) Strongly Disagree
B) Disagree
C) Neutral
D) Agree
E) Strongly Agree  
Your answer (A/B/C/D/E): c

Q4: Do you believe your vote can bring change, even if it's just one among millions?
A) Strongly Disagree
B) Disagree
C) Neutral
D) Agree
E) Strongly Agree  
Your answer (A/B/C/D/E): c

Q5: Do you trust the election process in your country to be fair?
A) Stro

In [11]:
# STEP 3: Build GPT prompt using the answers
prompt = (
    "You are a psychologist using the OCEAN personality model (Openness, Conscientiousness, Extraversion, Agreeableness, Neuroticism).\n"
    "Given the user's answers to 10 democracy-related questions, assign a score from 1 to 10 for each trait.\n"
    "Then identify the dominant trait and provide a short explanation.\n\n"
)

for i, (q, a) in enumerate(qa_pairs, 1):
    prompt += f"Q{i}: {q}\nAnswer: {a}\n\n"

prompt += (
    "Return the output in this format:\n"
    "- Openness: #\n"
    "- Conscientiousness: #\n"
    "- Extraversion: #\n"
    "- Agreeableness: #\n"
    "- Neuroticism: #\n\n"
    "Top Trait: [Trait Name]\n"
    "Explanation: [Short explanation of why this is dominant]"
)


In [12]:
# STEP 4: Send request to OpenRouter API (GPT-3.5)
OPENROUTER_API_KEY = "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"  # 🔐 Replace this securely

headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json"
}

data = {
    "model": "openai/gpt-3.5-turbo",
    "messages": [{"role": "user", "content": prompt}]
}

response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)

In [13]:
# STEP 5: Process GPT response
if response.status_code == 200:
    output = response.json()['choices'][0]['message']['content']
    print("\n GPT-3.5 Personality Analysis (Raw):\n")
    print(output)

      # STEP 6: Extract raw OCEAN scores (1–10)
    import re

    trait_list = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']
    raw_scores = {}

    for trait in trait_list:
        match = re.search(fr"{trait}:\s*(\d+)", output)
        if match:
            raw_scores[trait] = int(match.group(1))
        else:
            raw_scores[trait] = None

    # STEP 7: Detect all top traits
    max_raw_score = max(score for score in raw_scores.values() if score is not None)
    top_traits = [trait for trait, score in raw_scores.items() if score == max_raw_score]



    # STEP 9: Display top traits
    print(f"\n🌟 Top Trait{'s' if len(top_traits) > 1 else ''}: {', '.join(top_traits)}")

    # STEP 10: Policy suggestions
    policy_suggestions = {
        "Openness": (
            "You are a visionary thinker!\n"
            "- Explore the Digital India Act 2023: https://www.meity.gov.in/digital-india-act-2023\n"
            "- Participate in the Youth Parliament Scheme: https://nyps.mpa.gov.in/\n"
            "- Follow think tanks like Vidhi Legal Policy: https://vidhilegalpolicy.in/\n"
            "- Join civic hackathons and digital reform initiatives\n"
            "- Engage with MyGov for public consultations: https://www.mygov.in/"
        ),
        "Conscientiousness": (
            "You’re highly civic-minded and responsible!\n"
            "- Register to vote: https://www.nvsp.in/\n"
            "- Review MLA performance (Praja Foundation): https://praja.org/\n"
            "- Study the Union Budget: https://www.indiabudget.gov.in/\n"
            "- Volunteer for election monitoring\n"
            "- Use grievance portals like CPGRAMS: https://pgportal.gov.in/"
        ),
        "Extraversion": (
            " You thrive in social and group efforts!\n"
            "- Join Nehru Yuva Kendra: https://nyks.nic.in/\n"
            "- Organize or attend public consultations\n"
            "- Host voter awareness events\n"
            "- Join SVEEP initiatives: https://ecisveep.nic.in/\n"
            "- Lead constitutional literacy drives"
        ),
        "Agreeableness": (
            " Your compassion drives change!\n"
            "- Advocate for disability inclusion: https://www.orfonline.org/research/disability-inclusion-in-india/\n"
            "- Work with NGOs on social justice\n"
            "- Promote inclusive government schemes\n"
            "- Use CPGRAMS to help citizens: https://pgportal.gov.in/\n"
            "- Educate on minority and elderly rights"
        ),
        "Neuroticism": (
            " Your awareness protects democracy!\n"
            "- Report electoral violations with the cVIGIL App: https://play.google.com/store/apps/details?id=in.nic.eci.cvigil\n"
            "- File complaints on CPGRAMS: https://pgportal.gov.in/\n"
            "- Subscribe to fact-checking: https://altnews.in/\n"
            "- Use RTI to investigate issues: https://rtionline.gov.in/\n"
            "- Follow ECI alerts: https://eci.gov.in/"
        )
    }

    print("📢 Recommended Civic Engagement:")
    for trait in top_traits:
        suggestion = policy_suggestions.get(trait, "🔍 No suggestion found for this trait.")
        print(f"\n🧭 {trait}:\n{suggestion}")

else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)


❌ Error: 404
{"error":{"message":"No endpoints found for openai/gpt-3.5-turbo.","code":404},"user_id":"user_2uq6z2zBi2uMlPZ9t4t67xGnAYI"}
